# <center><u> Final LSTM/GRU Layout - Adri and Elian

In [186]:
#Import Libraries and Packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from dotenv import load_dotenv
load_dotenv()

True

In [187]:
#Access Credentials
db_host = os.getenv("DB_HOST")
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_port = os.getenv("DB_PORT")

In [188]:
conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

In [189]:
query="""
    select * 
    from agg.tidy_data_final
    where site = 20
"""

In [190]:
df = pd.read_sql_query(query, conn)

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_35932/1553155693.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [191]:
df = df.set_index('timestamp', drop=False)
df = df.sort_index()

# Prepare Data Set for LSTM/GRU Models

In [192]:
# Parameters

#n_input = no. of time steps considered before making the prediction
#n_features = no. of variables/features considered
#n_output = no. of time steps to forecast

# Note: 1 time step = 30 mins
# Look at the last two weeks to forecast the next 36 hours

# 1 day = 2 * 24 * 1 = 24
# 2 hours = 2 * 2 = 8 

n_input = 144
n_features = 14 #These includes the weather predictions that will be added later on and the normalised varaibles to be created
n_output = 72


# Selectfeatures of use for later on
df_input = df[['net_load', 'year', 'month', 'day', 'hour', 'day_of_week', 'season','precipitation_probability',
               'solar_radiation','sunshine_duration','weekend_or_bank_holiday','avg_net_load']]

# Normalise Data (as seen in Literature)

Group decision to use MaxAbs Scaler to normalise the data.

In [193]:
#Normalise all continuous variables needing scaling
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

variables = ['net_load','solar_radiation','sunshine_duration','avg_net_load']
for var in variables:
    new_var_name = var + '_norm'
    df_input[new_var_name] = scaler.fit_transform(df_input[[var]])

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_35932/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input[new_var_name] = scaler.fit_transform(df_input[[var]])
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_35932/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input[new_var_name] = scaler.fit_transform(df_input[[var]])
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_35932/4282254998.py:8: SettingWithCopyWarning: 
A value is trying to be

Now, we also want to include the predictions for 'solar radiation', 'sunshine_duration', 'precipitation_probability' as inputs.
These will be treated as new inputs (t+48) ~ 48 half-hourly timestamps representing day-ahead predictions.

In [194]:
df_input['sunshine_duration(t+48)'] = df_input['sunshine_duration_norm'].shift(-48) #Add prediction for time the day after
df_input['solar_radiation(t+48)'] = df_input['solar_radiation_norm'].shift(-48) #Add prediction for time the day after
df_input['precipitation_probability(t+48)'] = df_input['precipitation_probability'].shift(-48) #Add prediction for time the day after

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_35932/3763762080.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['sunshine_duration(t+48)'] = df_input['sunshine_duration_norm'].shift(-48) #Add prediction for time the day after
/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_35932/3763762080.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['solar_radiation(t+48)'] = df_input['solar_radiation_norm'].shift(-48) #Add prediction for time the day after
/var/folders/n

In [195]:
#We want to drop last 48 rows as they will have Na values for the (t+48) columns/inputs.
df_input.dropna(inplace=True)

/var/folders/n_/s_vk866s6ds93jqhjv__r6cr0000gn/T/ipykernel_35932/1212278532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input.dropna(inplace=True)


In [196]:
#Only Keep the Normalised Variables for the Model
df_input = df_input[['net_load_norm','year','month','day','hour','day_of_week','season','weekend_or_bank_holiday',
               'solar_radiation_norm','sunshine_duration_norm','avg_net_load_norm','sunshine_duration(t+48)',
               'solar_radiation(t+48)','precipitation_probability(t+48)']]

# Generate X and Y Matrices

In [197]:
X, y = [], []
for i in range(len(df_input) - n_input - n_output + 1):
    # Select all columns for the input sequence
    X.append(df_input.iloc[i:(i + n_input)].values)  
    
    y.append(df_input['net_load_norm'].iloc[(i + n_input):(i + n_input + n_output)].values)

X = np.array(X).reshape((len(X), n_input, -1))  
y = np.array(y)

# Train and Test Split

In [198]:
# Split into train (First ~330 days) and test (30 days)
# 1 time step = 30 mins

n_total_days = len(X) / (2 * 24)
n_test_days = 30
n_train_split = int((n_total_days - n_test_days) * 2 * 24)

X_train = X[:n_train_split]
y_train = y[:n_train_split]

X_test = X[n_train_split:]
y_test = y[n_train_split:]

# Random Search - Hyperparameter Tuning (LSTM/GRU)

In [199]:
# Import all Necessary Keras Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam as LegacyAdam

Hyperparameters to tune:
* \# of LSTM/GRU units
* \# of LSTM/GRU layers
* Dropout Rate
* Batch Size

In [174]:
import keras_tuner as kt

In [269]:
#Hyperparameter Tuning Function:
def build_model(hp):
   model = Sequential()
    #Input Layer
   model.add(GRU(units=hp.Int('gru_units_layer1', min_value=32, max_value=256, step=32),
                activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]),
                return_sequences=True))
   model.add(Dropout(hp.Float('dropout_rate1', min_value=0.1, max_value=0.5, step=0.1),seed = 12345))
   model.add(BatchNormalization())

    #Hidden GRU Layer Optimisation
   for i in range(hp.Int('num_middle_gru_layers', min_value=1, max_value=3)):
      model.add(GRU(units=hp.Int(f'gru_units_layer{i+1}', min_value=32, max_value=256, step=32),
                     activation='relu', return_sequences=True))
      model.add(Dropout(hp.Float(f'dropout_rate{i+1}', min_value=0.1, max_value=0.5, step=0.1)))
      model.add(BatchNormalization())
   model.add(GRU(units=hp.Int(f'gru_units_final', min_value=32, max_value=256, step=32),
                     activation='relu'))
    #Ouput Layer
   model.add(Dense(y_train.shape[1]))

   batch_size = hp.Choice('batch_size', values=[16, 32, 64, 128])

   model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=[MeanSquaredError(), MeanAbsoluteError(),RootMeanSquaredError()])
   return model

In [270]:
build_model(kt.HyperParameters())

In [271]:
rtuner = kt.RandomSearch(hypermodel=build_model,
    objective="mse",
    max_trials=5,
    seed=12345,
    overwrite=True,
    directory="my_dir",
    project_name="Hyperparameter Tuner")

In [272]:
rtuner.search(X_train, y_train, epochs=10, validation_data = (X_test,y_test))

Trial 2 Complete [02h 54m 33s]

Best mse So Far: None
Total elapsed time: 03h 28m 15s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
160               |128               |gru_units_layer1
0.4               |0.3               |dropout_rate1
1                 |2                 |num_middle_gru_layers
256               |32                |gru_units_final
32                |16                |batch_size
96                |32                |gru_units_layer2
0.4               |0.1               |dropout_rate2



Epoch 1/10
493/493 [==============================] - 1245s 3s/step - loss: 0.0264 - mean_squared_error: 0.0264 - mean_absolute_error: 0.1120 - root_mean_squared_error: 0.1626 - val_loss: 0.0359 - val_mean_squared_error: 0.0359 - val_mean_absolute_error: 0.1362 - val_root_mean_squared_error: 0.1894
Epoch 2/10
493/493 [==============================] - 1239s 3s/step - loss: 0.0158 - mean_squared_error: 0.0158 - mean_absolute_error: 0.0864 - root_mean_squared_error: 0.1257 - val_loss: 0.0271 - val_mean_squared_error: 0.0271 - val_mean_absolute_error: 0.1040 - val_root_mean_squared_error: 0.1645
Epoch 3/10
493/493 [==============================] - 1157s 2s/step - loss: 0.0135 - mean_squared_error: 0.0135 - mean_absolute_error: 0.0797 - root_mean_squared_error: 0.1160 - val_loss: 0.0285 - val_mean_squared_error: 0.0285 - val_mean_absolute_error: 0.1314 - val_root_mean_squared_error: 0.1687
Epoch 4/10
493/493 [==============================] - 221s 449ms/step - loss: 0.0120 - mean_squared_

Traceback (most recent call last):
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in <listcomp>
    [convert_to_metrics_dict(elem, objective) for elem in results]
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tu

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/base_tuner.py", line 265, in _run_and_update_trial
    tuner_utils.convert_to_metrics_dict(
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in convert_to_metrics_dict
    [convert_to_metrics_dict(elem, objective) for elem in results]
  File "/Users/adriachimenos/anaconda3/envs/tf/lib/python3.11/site-packages/keras_tuner/src/engine/tuner_utils.py", line 132, in <listcomp>
    [convert_to_metrics_dict(elem, objective) for elem in results]
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users

In [ ]:
rtuner.search_space.update({'batch_size': [32, 64, 128]})

In [ ]:
tuned_model = rtuner.get_best_models(num_models=1)

In [ ]:
tuned_model.summary()

In [ ]:
rtuner.get_best_hyperparameters().get_config()['values']